In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [3]:
data_dir = './dataset'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [4]:
model = models.mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.DEFAULT')
# Initialize model and optimizer
num_features = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_features, 10)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Declare optimizer after defining the model
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Loss function
criterion = nn.CrossEntropyLoss()

In [5]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")


Epoch 1/50
----------
train Loss: 1.3008 Acc: 0.6302
val Loss: 0.4867 Acc: 0.8600
Epoch 2/50
----------
train Loss: 0.3622 Acc: 0.8883
val Loss: 0.2363 Acc: 0.9400
Epoch 3/50
----------
train Loss: 0.2419 Acc: 0.9238
val Loss: 0.1550 Acc: 0.9640
Epoch 4/50
----------
train Loss: 0.1996 Acc: 0.9339
val Loss: 0.1340 Acc: 0.9600
Epoch 5/50
----------
train Loss: 0.1604 Acc: 0.9482
val Loss: 0.1280 Acc: 0.9600
Epoch 6/50
----------
train Loss: 0.1482 Acc: 0.9513
val Loss: 0.1010 Acc: 0.9690
Epoch 7/50
----------
train Loss: 0.1301 Acc: 0.9544
val Loss: 0.0893 Acc: 0.9690
Epoch 8/50
----------
train Loss: 0.1194 Acc: 0.9598
val Loss: 0.0966 Acc: 0.9690
Epoch 9/50
----------
train Loss: 0.1084 Acc: 0.9639
val Loss: 0.0904 Acc: 0.9690
Epoch 10/50
----------
train Loss: 0.1005 Acc: 0.9654
val Loss: 0.1083 Acc: 0.9690
Epoch 11/50
----------
train Loss: 0.0902 Acc: 0.9705
val Loss: 0.0797 Acc: 0.9740
Epoch 12/50
----------
train Loss: 0.0934 Acc: 0.9684
val Loss: 0.0869 Acc: 0.9670
Epoch 13/50
-

In [6]:
torch.save(model.state_dict(), 'model_weights.pth')
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi